In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
import cupy as cp
from sklearnex import patch_sklearn

In [2]:
def optimize_data_types(data):
    # Convert categorical columns to 'category' type
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    for col in categorical_columns:
        if col in data.columns:  # Check if the column exists
            data[col] = data[col].astype('category')
    
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include='number').columns:
        data[col] = pd.to_numeric(data[col], downcast='integer')
    
    return data

def mainBareBones():
    # Load data
    data = pd.read_csv('Loan Prediction.csv')

    # Drop rows with missing values
    data = data.dropna()

    # Optimize data types
    data = optimize_data_types(data)

    # Encode categorical data
    encoder = LabelEncoder()
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    for col in categorical_columns:
        if col in data.columns:  # Check if the column exists
            data[col] = encoder.fit_transform(data[col])

    # Split features (X) and target variable (y)
    target_column = 'Risk_Flag'  # Target variable
    X = data.drop(columns=[target_column])
    y = data[target_column]

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train AdaBoost Classifier
    model = AdaBoostClassifier(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate accuracy
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, zero_division=1))

if __name__ == "__main__":
    mainBareBones()


Evaluating the model...
Accuracy: 87.59%

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93     44147
           1       1.00      0.00      0.00      6253

    accuracy                           0.88     50400
   macro avg       0.94      0.50      0.47     50400
weighted avg       0.89      0.88      0.82     50400



In [3]:
def optimize_data_types(data):
    # Convert categorical columns to 'category' type
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    for col in categorical_columns:
        if col in data.columns:  # Check if the column exists
            data[col] = data[col].astype('category')
    
    # Downcast numerical columns to reduce memory usage with CuPy
    for col in data.select_dtypes(include='number').columns:
        cupy_array = cp.array(data[col], dtype=cp.float32)
        data[col] = cupy_array.get()
    
    return data

def mainCuPY():
    # Load data into a pandas DataFrame
    data = pd.read_csv('Loan Prediction.csv')

    # Drop rows with missing values
    data = data.dropna()

    # Optimize data types
    data = optimize_data_types(data)

    # Encode categorical data
    encoder = LabelEncoder()
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    for col in categorical_columns:
        if col in data.columns:  # Check if the column exists
            data[col] = encoder.fit_transform(data[col])

    # Convert features and target to CuPy arrays
    target_column = 'Risk_Flag'  # Target variable
    X = cp.array(data.drop(columns=[target_column]))
    y = cp.array(data[target_column])

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train AdaBoost Classifier
    model = AdaBoostClassifier(n_estimators=50, random_state=42)
    model.fit(cp.asnumpy(X_train), cp.asnumpy(y_train))  # Convert to NumPy for compatibility

    # Make predictions
    y_pred = model.predict(cp.asnumpy(X_test))  # Convert to NumPy for compatibility

    # Evaluate accuracy
    print("\nEvaluating the model...")
    accuracy = accuracy_score(cp.asnumpy(y_test), y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(cp.asnumpy(y_test), y_pred, zero_division=1))

if __name__ == "__main__":
    mainCuPY()



Evaluating the model...
Accuracy: 87.59%

Classification Report:
              precision    recall  f1-score   support

         0.0       0.88      1.00      0.93     44147
         1.0       1.00      0.00      0.00      6253

    accuracy                           0.88     50400
   macro avg       0.94      0.50      0.47     50400
weighted avg       0.89      0.88      0.82     50400



In [4]:
patch_sklearn() # Apply Intel optimizations

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
def optimize_data_types(data):
    # Convert categorical columns to 'category' type
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    for col in categorical_columns:
        if col in data.columns:  # Check if the column exists
            data[col] = data[col].astype('category')
    
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include='number').columns:
        data[col] = pd.to_numeric(data[col], downcast='integer')
    
    return data

def mainIntelex():
    # Load data
    data = pd.read_csv('Loan Prediction.csv')

    # Drop rows with missing values
    data = data.dropna()

    # Optimize data types
    data = optimize_data_types(data)

    # Encode categorical data
    encoder = LabelEncoder()
    categorical_columns = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE']
    for col in categorical_columns:
        if col in data.columns:  # Check if the column exists
            data[col] = encoder.fit_transform(data[col])

    # Split features (X) and target variable (y)
    target_column = 'Risk_Flag'  # Target variable
    X = data.drop(columns=[target_column])
    y = data[target_column]

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train AdaBoost Classifier
    model = AdaBoostClassifier(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate accuracy
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, zero_division=1))

if __name__ == "__main__":
    mainIntelex()


Evaluating the model...
Accuracy: 87.59%

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93     44147
           1       1.00      0.00      0.00      6253

    accuracy                           0.88     50400
   macro avg       0.94      0.50      0.47     50400
weighted avg       0.89      0.88      0.82     50400

